<a href="https://colab.research.google.com/github/kindustrii/doiit-text-klassifizierung-rnn/blob/main/doiit_text_klassifizierung_mit_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

Unser Neuronales Netz wir Filmbewertungen, die als Text vorliegen, darauf untersuchen ob es sich um eine positive oder um eine negative Bewertung handelt.

Als Trainingsdaten für das Neuronale Netz werden wird den IMDB-Datensatz verwenden. Es gibt mehrere Wege diesen Datensatz zu laden. Wir verwenden eine Variante mittleren Aufwandes und nutzen dafür das Package tensorflow-datasets. Da dieses Package nicht standardmäßig installiert ist, führen wir zunächst die Installation aus. Danach importierem notwendigen Bibliotheken.

In [1]:
!pip install -q tensorflow-datasets

In [2]:
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt

# IMDB-Datensatz laden

Der IMDB-Datensatz besteht aus 50.000 Filmbewertung (in englischer Sprache) mit je 25.000 positiven und negativen. Eine negative Bewertung ist mit einer 0 markiert und eine positive Bewertung ist mit einer 1 markiert.

In [3]:
# laden des gesamten Datensatzes
dataset, info = tfds.load('imdb_reviews',with_info=True,
                          as_supervised=True)

# Aufteilen des gesamten Datensatzen in Trainings- und Testdaten
# Die Testdaten werden zur Seite gelegt und wir verweden
# diese zum Schluss um das Neuronale Netz zu testen
# Wir teilen in diesem Fall den Datensatz in dem Verhältnis 50/50 auf
train_data, test_data = dataset['train'],dataset['test'] 


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9O92V9/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9O92V9/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete9O92V9/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


Sehen wir uns uns mal die ersten 5 Filmbewertung aus den Trainingsdaten an.

In [4]:
for bewertung,markierung in train_data.take(5):
  print('Bewertung:  {}'.format(bewertung.numpy()))
  print('Markierung: {}\n'.format(markierung.numpy()))

Bewertung:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Markierung: 0

Bewertung:  b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the fil

Das sieht doch ganz gut aus. Für die ersten fünf Filmbewertungen passt die Markierung 0 (negativ) und 1 (positiv) zu dem jeweiligen Text.

Diese Markierungen wurden von Menschen erstellt. D.h. ein Mensch hat sich die Bewertungen durchgelesen und je nachdem ob darin der Film gut oder schlecht bewertet wurde, mit einer 0 oder mit einer 1 markiert. 

Unsere Aufgabe besteht nun darin ein Neuronales Netz zu erstellen, das in der gleichen Weise wie der Mensch eine Filmbewertung bekommt und diese entweder mit einer 0 oder mit einer 1 markiert. Damit es diese Aufgabe erfolgreich durchführen kann, werden wir in das Neuronalen Netz die Trainingsdaten als Pärchen (Bewertung/Markierung) einspeisen. In dem Training wird, vereinfacht gesagt, das NN die Muster von positiven und negativen Bewertungen lernen und später eine unbekannte Filmbewertung auf diese Muster durchsuchen und so die Entscheidung treffen, ob mit einer 1 oder einer 0 Markiert werden soll.

In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [7]:
train_data = train_data.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
test_data = test_data.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [8]:
for bewertung, markierung in train_data.take(1):
  print('Bewertung: {}\n'.format(bewertung.numpy()[:3]))
  print('Markierung: {}\n'.format(markierung.numpy()[:3]))

Bewertung: [b'Who would\'ve imagined -- Hal Hartley creates a filmic corollary to Syriana while retaining his signature idiosyncratic style. The fusion is highly entertaining.<br /><br />Having not seen a Hal Hartley film for about a decade, I approached this one with some caution. His brilliant productions of the nineties had impressed critics and audiences with their unique style and dialog. The director\'s earlier films featured colorful characters and offered close observations of life -- often in the region of Long Island, New York or in New York City itself -- that were offbeat and insightful.<br /><br />My initial caution stemmed from the description of this movie as a "spy thriller". To my pleasant surprise, Hartley manages to mesh his well established style and focus to produce a highly original drama of international intrigue. It works in more ways than one might imagine. Hartley\'s film retains the dialog and character focus that are his trademarks, along with a singular cin